# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\Constipation_Processed_Files
df = pd.read_csv('ConstipationCateg_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head(2)

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\Constipation_Processed_Files
(458, 20757)


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_Constipate,cg13609937,cg13847987,cg13822231,cg20728472,...,cg06819251,cg22158158,cg20161227,cg05701765,cg25437180,cg06002203,cg21722170,cg13142750,cg12743359,cg04293703
0,200991620021_R01C01,3001,2,1,1,0,0.020100,0.922355,0.011956,0.021950,...,0.900701,0.906393,0.948309,0.847871,0.647087,0.836282,0.731078,0.62803,0.883242,0.534427
1,200973410167_R04C01,3007,2,1,1,0,0.071529,0.938088,0.013730,0.029759,...,0.739334,0.888118,0.914366,0.854601,0.685887,0.890144,0.816051,0.73572,0.892971,0.540466


# Removing possible duplicate entries (few CpGs were repeated having same data, names repeated has dot in it)
df1 = df.drop (columns = [col for col in df.columns if '.' in col])
print (df1.shape)
df1.to_csv ("ConstipationCateg_Methylome_APPRDXe.csv", index = False)

In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'MDSP_Constipate'], axis=1)
df1.head(2)

,APPRDX,cg13609937,cg13847987,cg13822231,cg20728472,cg14567798,cg14205952,cg13083110,cg24453271,cg18015632,...,cg06819251,cg22158158,cg20161227,cg05701765,cg25437180,cg06002203,cg21722170,cg13142750,cg12743359,cg04293703
0,1,0.020100,0.922355,0.011956,0.021950,0.019629,0.011816,0.531190,0.824585,0.879564,...,0.900701,0.906393,0.948309,0.847871,0.647087,0.836282,0.731078,0.62803,0.883242,0.534427
1,1,0.071529,0.938088,0.013730,0.029759,0.025837,0.013719,0.578083,0.820133,0.892031,...,0.739334,0.888118,0.914366,0.854601,0.685887,0.890144,0.816051,0.73572,0.892971,0.540466


In [4]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,448,449,450,451,452,453,454,455,456,457
APPRDX,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.0000,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000
cg13609937,0.0201,0.071529,0.068068,0.022575,0.010521,0.025196,0.017565,0.0187,0.024127,0.016457,...,0.018173,0.018913,0.018837,0.0176,0.020231,0.016197,0.023998,0.023032,0.02184,0.020266


In [5]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [6]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,APPRDX,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000,1.000000,...,2.000000,2.000000,2.000000,2.0000,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000
1,cg13609937,0.0201,0.071529,0.068068,0.022575,0.010521,0.025196,0.017565,0.0187,0.024127,...,0.018173,0.018913,0.018837,0.0176,0.020231,0.016197,0.023998,0.023032,0.02184,0.020266


In [7]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,cg13609937,0.020100,0.071529,0.068068,0.022575,0.010521,0.025196,0.017565,0.018700,0.024127,...,0.018173,0.018913,0.018837,0.017600,0.020231,0.016197,0.023998,0.023032,0.021840,0.020266
2,cg13847987,0.922355,0.938088,0.943100,0.949174,0.969717,0.958496,0.939032,0.960726,0.935071,...,0.943517,0.959274,0.955007,0.967183,0.963796,0.968448,0.970054,0.940215,0.927902,0.933645


In [8]:
df2 = pd.read_csv ("Constipation_Boruta_Top50.csv")
df2.head(2)

,CpG,Ranking
0,cg27281389,1
1,cg12298429,1


In [9]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Ranking
0,cg13332222,0.902238,0.928699,0.914653,0.937166,0.924741,0.938349,0.936708,0.936956,0.934214,...,0.919156,0.943549,0.923855,0.898360,0.924826,0.925401,0.926145,0.922367,0.929128,20
1,cg21043558,0.667449,0.659961,0.634282,0.568346,0.523774,0.703552,0.583571,0.510681,0.711800,...,0.670767,0.439230,0.660370,0.620377,0.574526,0.641473,0.624955,0.528597,0.484300,18


In [10]:
df3 = df3.drop(['Ranking'], axis=1)
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,cg13332222,0.902238,0.928699,0.914653,0.937166,0.924741,0.938349,0.936708,0.936956,0.934214,...,0.934987,0.919156,0.943549,0.923855,0.898360,0.924826,0.925401,0.926145,0.922367,0.929128
1,cg21043558,0.667449,0.659961,0.634282,0.568346,0.523774,0.703552,0.583571,0.510681,0.711800,...,0.718336,0.670767,0.439230,0.660370,0.620377,0.574526,0.641473,0.624955,0.528597,0.484300


In [11]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CpG,cg13332222,cg21043558,cg17755880,cg12298429,cg27281389,cg25434660,cg19924025,cg07581257,cg15069620,cg13818654,...,cg15649411,cg03478448,cg20187858,cg09233704,cg05142211,cg20567867,cg12683454,cg12131939,cg26458329,cg19371224
1.0,0.902238,0.667449,0.011988,0.032801,0.026921,0.84856,0.017226,0.856946,0.904181,0.06676,...,0.84894,0.636575,0.164275,0.620103,0.035018,0.920654,0.023826,0.907596,0.029374,0.751572


In [12]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [13]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg13332222,cg21043558,cg17755880,cg12298429,cg27281389,cg25434660,cg19924025,cg07581257,cg15069620,...,cg15649411,cg03478448,cg20187858,cg09233704,cg05142211,cg20567867,cg12683454,cg12131939,cg26458329,cg19371224
0,CpG,cg13332222,cg21043558,cg17755880,cg12298429,cg27281389,cg25434660,cg19924025,cg07581257,cg15069620,...,cg15649411,cg03478448,cg20187858,cg09233704,cg05142211,cg20567867,cg12683454,cg12131939,cg26458329,cg19371224
1,1.0,0.902238,0.667449,0.011988,0.032801,0.026921,0.84856,0.017226,0.856946,0.904181,...,0.84894,0.636575,0.164275,0.620103,0.035018,0.920654,0.023826,0.907596,0.029374,0.751572


In [14]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg13332222,cg21043558,cg17755880,cg12298429,cg27281389,cg25434660,cg19924025,cg07581257,cg15069620,...,cg15649411,cg03478448,cg20187858,cg09233704,cg05142211,cg20567867,cg12683454,cg12131939,cg26458329,cg19371224
1,1.0,0.902238,0.667449,0.011988,0.032801,0.026921,0.84856,0.017226,0.856946,0.904181,...,0.84894,0.636575,0.164275,0.620103,0.035018,0.920654,0.023826,0.907596,0.029374,0.751572
2,1.0,0.928699,0.659961,0.01022,0.042715,0.03342,0.864457,0.049359,0.912148,0.914791,...,0.888982,0.605065,0.122112,0.682452,0.041906,0.912083,0.02897,0.894917,0.021559,0.855625


In [15]:
df3.reset_index(inplace = True, drop = True)
df3.head()

,APPRDX,cg13332222,cg21043558,cg17755880,cg12298429,cg27281389,cg25434660,cg19924025,cg07581257,cg15069620,...,cg15649411,cg03478448,cg20187858,cg09233704,cg05142211,cg20567867,cg12683454,cg12131939,cg26458329,cg19371224
0,1.0,0.902238,0.667449,0.011988,0.032801,0.026921,0.84856,0.017226,0.856946,0.904181,...,0.84894,0.636575,0.164275,0.620103,0.035018,0.920654,0.023826,0.907596,0.029374,0.751572
1,1.0,0.928699,0.659961,0.01022,0.042715,0.03342,0.864457,0.049359,0.912148,0.914791,...,0.888982,0.605065,0.122112,0.682452,0.041906,0.912083,0.02897,0.894917,0.021559,0.855625
2,1.0,0.914653,0.634282,0.009464,0.033475,0.026175,0.882515,0.061998,0.910067,0.915184,...,0.906822,0.6937,0.137548,0.676492,0.019257,0.907489,0.031558,0.89922,0.021313,0.86027
3,1.0,0.937166,0.568346,0.0088,0.032138,0.019973,0.917281,0.027765,0.917015,0.935366,...,0.910821,0.723459,0.075339,0.755091,0.018673,0.930307,0.018857,0.933969,0.087466,0.858269
4,1.0,0.924741,0.523774,0.007376,0.030728,0.014377,0.887688,0.01242,0.880797,0.918228,...,0.878343,0.691091,0.08396,0.683017,0.024418,0.931743,0.014323,0.916397,0.028157,0.849493


In [16]:
df3.shape

(458, 51)

In [17]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)


## RanFor 1000 times, CpG only 

In [18]:
df3.head(2)

,APPRDX,cg13332222,cg21043558,cg17755880,cg12298429,cg27281389,cg25434660,cg19924025,cg07581257,cg15069620,...,cg15649411,cg03478448,cg20187858,cg09233704,cg05142211,cg20567867,cg12683454,cg12131939,cg26458329,cg19371224
0,1,0.902238,0.667449,0.011988,0.032801,0.026921,0.84856,0.017226,0.856946,0.904181,...,0.84894,0.636575,0.164275,0.620103,0.035018,0.920654,0.023826,0.907596,0.029374,0.751572
1,1,0.928699,0.659961,0.01022,0.042715,0.03342,0.864457,0.049359,0.912148,0.914791,...,0.888982,0.605065,0.122112,0.682452,0.041906,0.912083,0.02897,0.894917,0.021559,0.855625


In [19]:
df3.shape

(458, 51)

In [20]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [21]:
dframe1.to_csv ("Constipation_Top50CpG_Accuracy_1000RF.csv")

In [22]:
dframe1.head(3)

,Accuracy,Std_Dev
0,0.545,0.125399
1,0.625,0.055902
2,0.660,0.058310


## RanFor 1000 times, CpG with Severity (Normal, Mild, Severe) 

df4 = df3
df4 ['MDSP_Constipate'] = df ['MDSP_Constipate'].apply(int)

np.random.seed (1)
dframe2 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1000) :
        print(i)
        dataset = df4.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe2 = dframe2.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)
